In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
val_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
training_set = train_datagen.flow_from_directory('/kaggle/input/project/train/',
                                                 target_size = (100, 100),
                                                 batch_size = 10,
                                                 class_mode = 'binary')

In [ ]:
val_set = val_datagen.flow_from_directory('/kaggle/input/project/test/',
                                            target_size = (100, 100),
                                            batch_size = 8,
                                            class_mode = 'binary')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
local_weight_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
pretrained_model = InceptionV3(input_shape= (100,100,3),include_top = False,weights = None)

In [ ]:
pretrained_model.load_weights(local_weight_file)

In [ ]:
for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
pretrained_model.summary()

In [ ]:
last_layer = pretrained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
print('last layer output shape: ', last_layer.output_shape)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs.get('binary_accuracy')>=0.90):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(last_output)
x = layers.Dense(1024,activation = 'relu')(x)
x = layers.Dropout(.2)(x)
x = layers.Dense(1,activation ='sigmoid')(x)
model = Model(pretrained_model.input,x)

In [ ]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.BinaryCrossentropy(), 
    metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
model.summary()

In [ ]:
my_callback_object = myCallback()
history = model.fit_generator(
    training_set,
    epochs=100, verbose =1,validation_data=val_set,callbacks=[my_callback_object])

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [ ]:
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")